In [1]:
using Pkg
Pkg.activate("libs/")
using JLD2
using DICOM
using Dates
using XLSX
using Plots
using ImageView
using Images
using Statistics

BAC_root_dir = "/home/molloi-lab/Desktop/Project BAC/BAC_Full_Data"
excel_path = "/home/molloi-lab/Desktop/Project BAC/BAC project/Ca mass calculation Final.xlsx"
V_P_dict = Dict("L CC" => 6, "L MLO" => 2, "R CC" => 14, "R MLO" => 10)

image_dirs = ["Patientstudy_BAC_2013a/data_from_KP", 
"Patientstudy_BAC_2013b/data_from_KP", 
"Patientstudy_BAC_2014/data_from_KP", 
"Patientstudy_BAC_2015a/data_from_KP", 
"Patientstudy_BAC_2015b/data_from_KP", 
"Patientstudy_BAC_2016/Data from KP"]

save_dir = "/media/molloi-lab/System Files/BAC_dataset/Clean_Dataset_full"

patch_size = 256
patch_size_half = round(Int, patch_size/2);

  Activating project at `~/Desktop/Project BAC/BAC project/libs`


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/molloi-lab/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/molloi-lab/snap/code/common/.cache/gio-modules/libgiolibproxy.so


In [2]:
# @load "result_from_step1.jld" output_csv

# Helper functions

In [3]:
function zoom_pixel_values(img)
    a, b = minimum(img), maximum(img)
    img_ = (img .- a) ./ (b - a)
    return img_
end

zoom_pxiel_values (generic function with 1 method)

In [4]:
function get_num_of_patches(img, lbl, mask)
    s = size(img)
    x = ceil(Int, s[1]/patch_size) + floor(Int, (s[1]-patch_size_half)/patch_size)
    y = ceil(Int, s[2]/patch_size) + floor(Int, (s[2]-patch_size_half)/patch_size)
    ct_empty, ct_non_empty = 0, 0
    for i = 1 : x-1
        x_start = 1+(i-1)*patch_size_half
        x_end = x_start+patch_size-1
        for j = 1 : y-1
            y_start = 1+(j-1)*patch_size_half
            y_end = y_start+patch_size-1
            # check if this is forgounrd
            if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
                # check if contains BAC
                if sum(lbl[x_start:x_end, y_start:y_end]) > 0
                    ct_non_empty += 1
                else
                    ct_empty += 1
                end
            end
        end
        # right col
        y_start, y_end = s[2]-patch_size+1, s[2]
        # check if this is forgounrd
        if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
            # check if contains BAC
            if sum(lbl[x_start:x_end, y_start:y_end]) > 0
                ct_non_empty += 1
            else
                ct_empty += 1
            end
        end
    end
    # last row
    x_start, x_end = s[1]-patch_size+1, s[1]
    for j = 1 : y-1
        y_start = 1+(j-1)*patch_size_half
        y_end = y_start+patch_size-1
        # check if this is forgounrd
        if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
            # check if contains BAC
            if sum(lbl[x_start:x_end, y_start:y_end]) > 0
                ct_non_empty += 1
            else
                ct_empty += 1
            end
        end
    end
    # right col
    y_start, y_end = s[2]-patch_size+1, s[2]
    # check if this is forgounrd
    if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
        # check if contains BAC
        if sum(lbl[x_start:x_end, y_start:y_end]) > 0
            ct_non_empty += 1
        else
            ct_empty += 1
        end
    end
    return ct_empty, ct_non_empty
end

get_num_of_patches (generic function with 1 method)

# Finish dataset

In [12]:
# runtime ~= 20 mins
SIDs = readdir(save_dir)
num_SIDs = size(SIDs)[1]
ct_array = zeros(num_SIDs, 2)
Threads.@threads for i = 1 : num_SIDs
    SID = SIDs[i]
    files = readdir(joinpath(save_dir, SID))
    for f in files
        f_name, f_ext = splitext(f)
        if f_ext == ".dcm"
            # dcm
            splited = split(f, '.')
            if size(splited)[1] > 3
                ct_empty, ct_non_empty = split(splited[3], '_')
                ct_array[i, 1] += parse(Int, ct_empty)
                ct_array[i, 2] += parse(Int, ct_non_empty)
            else
                # set path
                old_img_path = joinpath(save_dir, SID, f)
                old_lbl_path = joinpath(save_dir, SID, f_name*".png")
                # read dicom image
                img = dcm_parse(old_img_path)[(0x7fe0, 0x0010)]
                # read label
                lbl = Images.load(old_lbl_path)
                # get mask
                mask = 1f0 .- round.(zoom_pixel_values(Float32.(img)))
                # get num of batch
                ct_empty, ct_non_empty = get_num_of_patches(img, lbl, mask)
                ct_array[i, 1] += ct_empty
                ct_array[i, 2] += ct_non_empty
                # set new name
                new_lbl_path = joinpath(save_dir, SID, f_name*".$(ct_empty)_$(ct_non_empty).png")
                new_img_path = joinpath(save_dir, SID, f_name*".$(ct_empty)_$(ct_non_empty).dcm")
                # rename
                mv(old_img_path, new_img_path)
                mv(old_lbl_path, new_lbl_path)
            end
        elseif f_ext == ".roi"
            # roi
            rm(joinpath(save_dir, SID, f))
        end
    end
    # GC.gc(true)
end
total_ct_empty, total_ct_non_empty = sum(ct_array[:, 1]), sum(ct_array[:, 2])

(5.122436e6, 103085.0)

Total number of BAC patches = 103085  
Total number of wihout BAC patches = 5122436

|      | img_ct | non_BAC  | BAC  |
|------|--------|----------|------|
| LCC  |  5111  | 1200903  | 23272|
| LMLO |  5116  | 1376274  | 26455|
| RCC  |  5112  | 1173377  | 24514|
| RMLO |  5113  | 1371882  | 28844|